In [2]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='93e131fb-ec0a-4d58-94c8-ebb0ae8d8c6e'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/b781fa29-d5c4-48a1-8460-71dc403491e7/wml_data/795c2169-f05e-46d8-a74c-369842cf8765/data/automl',
        model_location='auto_ml/b781fa29-d5c4-48a1-8460-71dc403491e7/wml_data/795c2169-f05e-46d8-a74c-369842cf8765/data/automl/model.zip',
        training_status='auto_ml/b781fa29-d5c4-48a1-8460-71dc403491e7/wml_data/795c2169-f05e-46d8-a74c-369842cf8765/training-status.json'
    )
)

Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [3]:
experiment_metadata = dict(
    prediction_type='binary',
    prediction_column='churn',
    holdout_size=0.1,
    scoring='accuracy',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=4,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    deployment_url='https://eu-gb.ml.cloud.ibm.com',
    project_id='2d85abbb-9a0d-4da2-b76e-202df7f5aabe',
    positive_label='True',
    drop_duplicates=True
)

## Set `n_jobs` parameter to the number of available CPUs

In [4]:
import os, ast
CPU_NUMBER = 1
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action:** Please provide IBM Cloud apikey following [docs](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [5]:
api_key = 'JdLCO_t6QthTYecu227qnTpSdTGqVE3BmqCTefAha5Dk'

In [6]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [7]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0]._wml_client = wml_client



<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`. It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`.

In [8]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True)

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [9]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA2
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from sklearn.ensemble import RandomForestClassifier

#### Pre-processing & Estimator.

In [10]:
column_selector_0 = ColumnSelector(
    columns_indices_list=[
        0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    ]
)
numpy_column_selector_0 = NumpyColumnSelector(columns=[0, 2, 3, 4, 5, 16, 18])
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str", "int_num", "char_str", "char_str", "int_num", "int_num",
        "int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], [], [], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
)
pipeline_0 = make_pipeline(
    column_selector_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
column_selector_1 = ColumnSelector(
    columns_indices_list=[
        0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    ]
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "int_num", "float_num", "int_num", "float_num", "float_num",
        "int_num", "float_num", "float_num", "int_num", "float_num",
        "float_num", "float_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    column_selector_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 2, 3, 4, 5, 16, 18, 1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,
    ],
)
ta2_0 = TA2(
    fun=np.add,
    name="sum",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "state", "account length", "area code", "international plan",
        "voice mail plan", "number vmail messages", "total day minutes",
        "total day calls", "total day charge", "total eve minutes",
        "total eve calls", "total eve charge", "total night minutes",
        "total night calls", "total night charge", "total intl minutes",
        "total intl calls", "total intl charge", "customer service calls",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 19),
    additional_col_count_to_keep=15,
    ptype="classification",
)
ta2_1 = TA2(
    fun=np.multiply,
    name="product",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "state", "account length", "area code", "international plan",
        "voice mail plan", "number vmail messages", "total day minutes",
        "total day calls", "total day charge", "total eve minutes",
        "total eve calls", "total eve charge", "total night minutes",
        "total night calls", "total night charge", "total intl minutes",
        "total intl calls", "total intl charge", "customer service calls",
        "sum(state__total day minutes)",
        "sum(total day minutes__total day calls)",
        "sum(total day minutes__total day charge)",
        "sum(total day minutes__total eve minutes)",
        "sum(total day minutes__total eve calls)",
        "sum(total day minutes__total eve charge)",
        "sum(total day minutes__total night calls)",
        "sum(total day minutes__total night charge)",
        "sum(total day minutes__total intl minutes)",
        "sum(total day minutes__total intl calls)",
        "sum(total day minutes__total intl charge)",
        "sum(total day charge__total eve charge)",
        "sum(total day charge__total night charge)",
        "sum(total day charge__total intl minutes)",
        "sum(total day charge__total intl charge)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 19),
    additional_col_count_to_keep=15,
    ptype="classification",
)
random_forest_classifier = RandomForestClassifier(
    class_weight="balanced",
    criterion="entropy",
    max_depth=5,
    max_features=0.9915796247496014,
    min_samples_leaf=2,
    min_samples_split=5,
    n_estimators=91,
    n_jobs=CPU_NUMBER,
    random_state=33,
)


#### Pipeline.

In [11]:
pipeline = make_pipeline(
    union,
    numpy_permute_array,
    ta2_0,
    fs1_0,
    ta2_1,
    fs1_1,
    random_forest_classifier,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [12]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [13]:
pipeline.fit(train_X.values, train_y.values);

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [14]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

0.9730538922155688


In [16]:
test_X

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
2169,IL,156.0,415.0,343-3296,no,no,0.0,174.5,65.0,29.67,197.4,116.0,16.78,238.5,86.0,10.73,10.6,2.0,2.86,0.0
774,OR,80.0,415.0,391-8087,no,no,0.0,161.1,99.0,27.39,198.8,81.0,16.90,228.4,116.0,10.28,10.6,4.0,2.86,1.0
1601,AR,99.0,510.0,387-2604,yes,no,0.0,242.3,102.0,41.19,350.9,102.0,29.83,163.1,93.0,7.34,11.3,3.0,3.05,0.0
516,MS,96.0,510.0,420-5990,no,no,0.0,98.2,100.0,16.69,307.2,88.0,26.11,182.5,120.0,8.21,7.6,1.0,2.05,2.0
251,NJ,106.0,415.0,365-2153,no,no,0.0,207.9,91.0,35.34,172.0,109.0,14.62,191.8,143.0,8.63,14.4,7.0,3.89,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,NY,137.0,510.0,338-7955,no,no,0.0,109.8,120.0,18.67,230.5,86.0,19.59,255.8,103.0,11.51,11.9,6.0,3.21,1.0
1240,TN,108.0,510.0,356-8449,no,yes,34.0,162.1,83.0,27.56,171.8,117.0,14.60,259.8,76.0,11.69,9.6,3.0,2.59,4.0
582,NM,104.0,415.0,356-7217,no,no,0.0,164.2,109.0,27.91,155.4,90.0,13.21,168.9,117.0,7.60,10.7,8.0,2.89,1.0
278,AL,131.0,415.0,361-7998,no,yes,25.0,192.7,85.0,32.76,225.9,105.0,19.20,254.2,59.0,11.44,10.9,6.0,2.94,2.0


In [17]:
#pipeline.predict(test_X.values)
import pandas  as pd
import numpy as np
#pd.DataFrame(test_X)

In [48]:
data = {"state":"NY","account length":161,
                               "area code":415,"phone number" :"351-7269",
                               "international plan": "no","voice mail plan": "no",
                               "number vmail messages":0,"total day minutes":332.9,
                               "total day calls":67,"total day charge": 56.59,
                               "total eve minutes": 317.8,"total eve calls":97,"total eve charge": 27.01,
                               "total night minutes": 160.6,"total night calls": 128,"total night charge":7.23,
                               "total intl minutes": 5.4,"total intl calls":9,"total intl charge": 1.46,
                               "customer service calls":4}
data_array = list(data.values())
an_array = np.array(data_array)
an_array = np.reshape(an_array,(1,an_array.size))
an_array

array([['NY', '161', '415', '351-7269', 'no', 'no', '0', '332.9', '67',
        '56.59', '317.8', '97', '27.01', '160.6', '128', '7.23', '5.4',
        '9', '1.46', '4']], dtype='<U32')

In [49]:
pipeline.predict(an_array)

array(['True'], dtype=object)

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2022 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___